V2 of the the GCN that uses Adam instead of handmade gradient descent as well as negative-log-likelihood loss. Consider adding dropout and biases to fully match paper. Scores 79% accuracy on CORA <br>
<br>
In Cora, nodes are documents and edges are citation links.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.optim as optim

import numpy as np
import scipy.sparse as sp

In [2]:
class GCN_layer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCN_layer, self).__init__()
        # I divide by in_feats to prevent the vanishing/exploding gradient problem
        # not a robust/scientific solution
        self.weights = Parameter(torch.randn(in_feats, out_feats).double()/in_feats)

    def forward(self, adj, feats):
        return adj @ feats @ self.weights

In [3]:
class GCN(nn.Module):
    def __init__(self, adj_list, out_labels, nl=3, init_feats=None, train_test=0.5):
        super(GCN, self).__init__()
        self.adj_list = torch.from_numpy(adj_list).double()
        sqrt = True # turn to False to use 'simple' adj_list normalization
        D = GCN.create_D(self.adj_list, sqrt=sqrt)
        if sqrt:
            self.norm_adj_list = D @ self.adj_list @ D # https://tkipf.github.io/graph-convolutional-networks/#fn3
        else:
            self.norm_adj_list = D @ self.adj_list
            
        self.out_labels = torch.from_numpy(out_labels).view(-1)
        self.nc = len(torch.unique(self.out_labels))
        
        # if no features given, use identity matrix
        if init_feats is None:
            init_feats = np.eye(len(adj_list))
        self.init_feats = torch.from_numpy(init_feats).double()
        
        # can give train indices directly or give a train/test split
        if type(train_test) == np.ndarray:
            self.train_indices = train_test
        else:
            self.train_indices = np.random.choice(np.arange(len(adj_list)), int(train_test*len(adj_list)), replace=False)
        
        # initialize layers
        self.gc1 = GCN_layer(init_feats.shape[1], 30)
        self.gc2 = GCN_layer(30, self.nc)
        
        # optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=1e-2)
        self.optimizer.zero_grad()
        
        self.epoch_stats = []
        
        
    def create_D(adj_list, sqrt=True):
        # creates the matrix for making A_hat
        # sqrt is optional, can simply invert the diagonal as well, but the paper recommends this approach
        D = torch.eye(len(adj_list), dtype=torch.double)
        diag_sums = adj_list.sum(dim=0)
        for i in range(len(adj_list)):
            if sqrt:
                D[i,i] = 1/torch.sqrt(diag_sums[i])
            else:
                D[i,i] = 1/diag_sums[i]
        return D
        
        
    def train(self, epochs=25):
        for _ in range(epochs):
            self.train_epoch()
        
        
    def train_epoch(self):
        """
        Propogates the model through the GCN using the formula:
            L_next = A_hat @ L_cur @ w_cur
            where A_hat is the normalized adj matrix
        Then, backprops the model and updates the weights
        Stores, the loss in self.epoch_stats
        """ 
        out = self.gc1(self.adj_list, self.init_feats)
        out = torch.relu(out)
        out = self.gc2(self.adj_list, out)
        out = torch.softmax(out, dim=1)
                
        loss = F.nll_loss(out[self.train_indices], self.out_labels[self.train_indices])
        loss.backward()
        
        self.optimizer.step()
        self.optimizer.zero_grad()
        
        self.epoch_stats.append(loss)
        
    
    def predict(self):
        with torch.no_grad():
            out = self.gc1(self.adj_list, self.init_feats)
            out = torch.relu(out)
            out = self.gc2(self.adj_list, out)
            out = torch.softmax(out, dim=1)
            return torch.argmax(out, dim=1)

In [4]:
import networkx

In [5]:
G = networkx.karate_club_graph()

In [6]:
adj_dict = dict(G.adjacency())

In [7]:
adj_m = np.eye(len(adj_dict.keys()))
for k in adj_dict:
    for n in adj_dict[k]:
        adj_m[k,n] = 1

In [8]:
adj_m # this has self loops, so A is a neighbor of A; this is good

array([[1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [9]:
out_labels = []
for d in G.nodes:
    if G.nodes[d]['club'] == 'Mr. Hi':
        out_labels.append(0)
    else:
        out_labels.append(1)
out_labels = np.array(out_labels)

In [10]:
out_labels # 0 = Mr. Hi, 1 = the other guy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [11]:
# giving the model about 0.3 of the data makes it predict nearly every node correctly
gcn = GCN(adj_m, out_labels, nl=3, init_feats=None, train_test=0.2)

In [12]:
preds_init = gcn.predict(); preds_init # nonsense initial predictions

tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [13]:
out_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
(preds_init.numpy() == out_labels).mean() # very bad clearly

0.08823529411764706

In [15]:
gcn.train(epochs=40)

In [16]:
preds = gcn.predict()
(preds.numpy() == out_labels).mean()

0.9705882352941176

In [17]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [18]:
# from https://github.com/tkipf/gcn
def load_data(path="data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    
    labels = np.where(labels)[1]

    return adj, features, labels, idx_train, idx_val, idx_test

In [19]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [20]:
adj = adj.todense(); adj.shape

(2708, 2708)

In [21]:
features = features.todense(); features

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [22]:
labels

array([4, 5, 6, ..., 1, 2, 4])

In [23]:
idx_train

range(0, 140)

In [24]:
known = np.array(list(idx_train)); known

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139])

In [25]:
gcn = GCN(adj, labels, init_feats=features, train_test=known)

In [26]:
init_preds = gcn.predict(); init_preds

tensor([1, 2, 4,  ..., 4, 2, 1])

In [27]:
(init_preds.numpy() == labels).mean()

0.1894387001477105

In [28]:
%time gcn.train(150)

CPU times: user 3min 48s, sys: 2.76 s, total: 3min 51s
Wall time: 21.5 s


In [29]:
preds = gcn.predict()
(preds.numpy() == labels).mean()

0.7950516986706057